In [100]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score


## Atividade 1

>**Implemente uma classe que corresponda ao SVM**

In [101]:
class SVM:
    def __init__(self, kernel='linear', C=1.0, gamma=None, degree=3, coef0=1, learning_rate=0.001, n_iters=1000):
        self.kernel = kernel
        self.C = C
        self.gamma = gamma
        self.degree = degree
        self.coef0 = coef0
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.w = None
        self.b = None
        self.alpha = None
        self.K = None

    def _kernel(self, x1, x2):
        if self.kernel == 'linear':
            return np.dot(x1, x2)
        elif self.kernel == 'poly':
            return (np.dot(x1, x2) + self.coef0) ** self.degree
        elif self.kernel == 'rbf':
            return np.exp(-self.gamma * np.linalg.norm(x1 - x2) ** 2)
        else:
            raise ValueError(f"Unsupported kernel type: {self.kernel}")

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.alpha = np.zeros(n_samples)
        self.b = 0
        y_ = np.where(y <= 0, -1, 1)

        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i, j] = self._kernel(X[i], X[j])

        for _ in range(self.n_iters):
            for i in range(n_samples):
                gradient = 1 - y_[i] * (np.sum(self.alpha * y_ * self.K[:, i]) + self.b)
                if self.alpha[i] < self.C:
                    self.alpha[i] += self.learning_rate * gradient

        if self.kernel == 'linear':
            self.w = np.dot(X.T, self.alpha * y_)
            self.b = np.mean(y_ - np.dot(self.K, self.alpha * y_))

    def project(self, X):
        if self.kernel == 'linear':
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(X.shape[0])
            for i in range(X.shape[0]):
                sum_alpha_kernel = np.sum(self.alpha * np.array([self._kernel(X[i], X_train[j]) for j in range(X_train.shape[0])]))
                y_predict[i] = sum_alpha_kernel
            return y_predict + self.b



    def predict(self, X):
        return np.sign(self.project(X))

    def evaluate(self, X_test, y_test):
        y_predict = self.predict(X_test)
        accuracy = np.mean(y_predict == y_test)
        return accuracy
   

## **Atividade 2 - Classificação com o dataset Iris**

Features: Total de 4

Para essa atividade utilizar as features:
1. Comprimento do sépalo (sepal length)
2. Largura do sépalo (sepal width)
3. Comprimento da pétala (petal length)
4. Largura da pétala (petal width)

Variável dependente: Espécie da flor Iris (Setosa, Versicolor ou Virginica)


>**Treine os dados da base Iris utilizando SVM implementado**

>**Faça as predições nos dados de teste e compare os dados reais com os preditos**

In [102]:
# Carregando o conjunto de dados Iris
data = load_iris()
colunas = ['sepal length', 'sepal width', 'petal length', 'petal width']
pd.DataFrame(data['data'], columns=colunas)

,sepal length,sepal width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [151]:
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Função para treinar e avaliar o modelo SVM no esquema one-vs-rest
def train_and_evaluate_svm(X_train, y_train, X_test, y_test):
    svms = []
    for i in np.unique(y_train):
        y_train_binary = np.where(y_train == i, 1, -1)
        svm = SVM(kernel='linear')
        svm.fit(X_train, y_train_binary)
        
        svms.append(svm)

    # Predizer as classes no conjunto de teste
    y_pred = np.zeros(X_test.shape[0])
    confidences = np.zeros((X_test.shape[0], len(svms)))
    for i, svm in enumerate(svms):
        confidences[:, i] = svm.project(X_test)
    y_pred = np.argmax(confidences, axis=1)

    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=iris.target_names))
    return y_pred

print("Avaliação com dados não normalizados:")
y_pred = train_and_evaluate_svm(
    X_train, y_train, X_test, y_test)
# Comparar os dados reais com os preditos
print("Real:    ", y_test)
print("Predito: ", y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n Acurácia do modelo: {accuracy * 100:.2f}%")


Avaliação com dados não normalizados:
Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        19
  versicolor       0.80      0.62      0.70        13
   virginica       0.69      0.85      0.76        13

    accuracy                           0.84        45
   macro avg       0.83      0.82      0.82        45
weighted avg       0.85      0.84      0.84        45

Real:     [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
Predito:  [1 0 1 2 1 0 2 2 1 1 2 0 0 0 0 2 2 1 1 2 0 2 0 2 2 2 1 2 0 0 0 0 1 0 0 2 2
 0 0 0 2 2 1 0 0]

 Acurácia do modelo: 84.44%


>**Normalize os dados da base Iris e refaça o treinamento do modelo**

Compare os resultados dos dados normalizados e não normalizados. Como métricas de avaliação podem utilizar recall, precision e f-measure.

In [106]:
# Normalizar os dados
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Treinar e avaliar o modelo com dados normalizados
print("\nAvaliação com dados normalizados:")
y_pred_normalized = train_and_evaluate_svm(X_train_normalized, y_train, X_test_normalized, y_test)



Avaliação com dados normalizados:
Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        19
  versicolor       0.67      0.46      0.55        13
   virginica       0.59      0.77      0.67        13

    accuracy                           0.78        45
   macro avg       0.75      0.74      0.74        45
weighted avg       0.78      0.78      0.77        45



In [ ]:
# Comparar os resultados dos dados normalizados e não normalizados
def compare_results(y_test, y_pred_non_normalized, y_pred_normalized):
    print("\nComparação de Resultados:")

    print("\nDados Não Normalizados:")
    precision_non_normalized = precision_score(y_test, y_pred_non_normalized, average='weighted')
    recall_non_normalized = recall_score(y_test, y_pred_non_normalized, average='weighted')
    f1_non_normalized = f1_score(y_test, y_pred_non_normalized, average='weighted')
    accuracy_non_normalized = accuracy_score(y_test, y_pred_non_normalized)
    
    print(f"Precision: {precision_non_normalized*100:.2f}%")
    print(f"Recall: {recall_non_normalized*100:.2f}%")
    print(f"F1-Score: {f1_non_normalized*100:.2f}%")
    print(f"Accuracy: {accuracy_non_normalized*100:.2f}%")

    print("\nDados Normalizados:")
    precision_normalized = precision_score(y_test, y_pred_normalized, average='weighted')
    recall_normalized = recall_score(y_test, y_pred_normalized, average='weighted')
    f1_normalized = f1_score(y_test, y_pred_normalized, average='weighted')
    accuracy_normalized = accuracy_score(y_test, y_pred_normalized)
    
    print(f"Precision: {precision_normalized*100:.2f}%")
    print(f"Recall: {recall_normalized*100:.2f}%")
    print(f"F1-Score: {f1_normalized*100:.2f}%")
    print(f"Accuracy: {accuracy_normalized*100:.2f}%")

compare_results(y_test, y_pred, y_pred_normalized)


Comparação de Resultados:

Dados Não Normalizados:
Precision: 85.19%
Recall: 84.44%
F1-Score: 84.23%
Accuracy: 84.44%

Dados Normalizados:
Precision: 78.47%
Recall: 77.78%
F1-Score: 77.24%
Accuracy: 77.78%


>**Divida o dataset em treinamento, teste e validação. Escreva um código que escolhe os melhores hiperparâmetros ajustando o conjunto de validação.**

Por exemplo qual o melhor Kernel vizinho (linear, poly, rbf), qual a melhor C, gamma. Como métricas de avaliação podem utilizar recall, precision e f-measure.

In [ ]:
# Carregar o dataset Iris
iris = load_iris()
X = iris.data
y = iris.target

# Mapear rótulos para binário: Setosa (-1) vs. não-Setosa (1)
y_binary = np.where(y == 0, -1, 1)

# Normalizar as features
scaler = StandardScaler()
X = scaler.fit_transform(X)


# Dividir o dataset em treinamento, teste e validação
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y_binary, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2


# Definir listas de valores para os hiperparâmetros a serem testados
kernels = ['linear', 'poly', 'rbf']
Cs = [0.1, 1, 10]
gammas = [0.1, 1, 10]

best_score = -1
best_params = None

# Loop para testar todas as combinações de hiperparâmetros
for kernel in kernels:
    for C in Cs:
        for gamma in gammas:
            # Instanciar e treinar o modelo SVM com os hiperparâmetros atuais
            svm = SVM(kernel=kernel, C=C, gamma=gamma,
                      learning_rate=0.001, n_iters=1000)
            svm.fit(X_train, y_train)

            # Avaliar no conjunto de validação
            y_pred_val = svm.predict(X_val)
            f1 = f1_score(y_val, y_pred_val, average='weighted')

            # Calcular outras métricas se necessário (precision, recall, f-measure)
            accuracy = accuracy_score(y_val, y_pred_val)
            precision = precision_score(y_val, y_pred_val)
            recall = recall_score(y_val, y_pred_val)
            f_measure = f1_score(y_val, y_pred_val)

            # Exibir os resultados para cada combinação de hiperparâmetros
            print(f"Kernel: {kernel}, C: {C}, gamma: {gamma}")
            print(
                f"Acurácia: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F-measure: {f_measure:.2f}")
            print("")

            # Verificar se é a melhor combinação de hiperparâmetros até agora
            if f1 > best_score:
                best_score = f1
                best_params = {'kernel': kernel, 'C': C, 'gamma': gamma}

# Usar os melhores hiperparâmetros encontrados para treinar o modelo final
best_model = SVM(**best_params)
best_model.fit(X_train, y_train)

# Avaliar no conjunto de teste
test_accuracy = best_model.evaluate(X_test, y_test)
print(f"Melhores hiperparâmetros: {best_params}")
print(f"Acurácia no conjunto de teste: {test_accuracy*100:.2f}%")

In [152]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Função para treinar e avaliar o modelo SVM no esquema one-vs-rest
def train_and_evaluate_svm_with_hyperparams(X_train, y_train, X_val, y_val, kernel, C, gamma):
    # Inicializar SVMs one-vs-rest para cada classe
    svms = []
    for i in np.unique(y_train):
        y_train_binary = np.where(y_train == i, 1, -1)
        svm = SVM(kernel=kernel, C=C, gamma=gamma)
        svm.fit(X_train, y_train_binary)
        svms.append(svm)

    # Predizer as classes no conjunto de validação
    y_val_pred = np.zeros(X_val.shape[0])
    confidences = np.zeros((X_val.shape[0], len(svms)))
    for i, svm in enumerate(svms):
        confidences[:, i] = svm.project(X_val)
    y_val_pred = np.argmax(confidences, axis=1)

    # Calcular métricas
    precision = precision_score(y_val, y_val_pred, average='weighted')
    recall = recall_score(y_val, y_val_pred, average='weighted')
    f1 = f1_score(y_val, y_val_pred, average='weighted')
    accuracy = accuracy_score(y_val, y_val_pred)
    
    # Exibir os resultados para cada combinação de hiperparâmetros testada
    print(f"Kernel: {kernel}, C: {C}, gamma: {gamma}")
    print(f"Acurácia: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")
    print("")

# Definir a grade de parâmetros para busca
param_grid = {
    'kernel': ['linear', 'poly', 'rbf'],
    'C': [0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1]
}

# Dividir os dados em conjuntos de treinamento, validação e teste
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Normalizar os dados
X_train_normalized = scaler.fit_transform(X_train)
X_val_normalized = scaler.transform(X_val)
X_test_normalized = scaler.transform(X_test)

# Realizar a busca pelos melhores hiperparâmetros
for kernel in param_grid['kernel']:
    for C in param_grid['C']:
        for gamma in param_grid['gamma']:
            train_and_evaluate_svm_with_hyperparams(
                X_train_normalized, y_train, X_val_normalized, y_val, kernel, C, gamma)


Kernel: linear, C: 0.1, gamma: 0.001
Acurácia: 0.80, Precision: 0.89, Recall: 0.80, F1-Score: 0.80

Kernel: linear, C: 0.1, gamma: 0.01
Acurácia: 0.80, Precision: 0.89, Recall: 0.80, F1-Score: 0.80

Kernel: linear, C: 0.1, gamma: 0.1
Acurácia: 0.80, Precision: 0.89, Recall: 0.80, F1-Score: 0.80

Kernel: linear, C: 1, gamma: 0.001
Acurácia: 0.83, Precision: 0.87, Recall: 0.83, F1-Score: 0.83

Kernel: linear, C: 1, gamma: 0.01
Acurácia: 0.83, Precision: 0.87, Recall: 0.83, F1-Score: 0.83

Kernel: linear, C: 1, gamma: 0.1
Acurácia: 0.83, Precision: 0.87, Recall: 0.83, F1-Score: 0.83

Kernel: linear, C: 10, gamma: 0.001
Acurácia: 0.77, Precision: 0.83, Recall: 0.77, F1-Score: 0.77

Kernel: linear, C: 10, gamma: 0.01
Acurácia: 0.77, Precision: 0.83, Recall: 0.77, F1-Score: 0.77

Kernel: linear, C: 10, gamma: 0.1
Acurácia: 0.77, Precision: 0.83, Recall: 0.77, F1-Score: 0.77

Kernel: poly, C: 0.1, gamma: 0.001
Acurácia: 0.60, Precision: 0.77, Recall: 0.60, F1-Score: 0.48

Kernel: poly, C: 0.1

c:\Users\Pichau\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Kernel: poly, C: 1, gamma: 0.001
Acurácia: 0.50, Precision: 0.35, Recall: 0.50, F1-Score: 0.41



c:\Users\Pichau\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Kernel: poly, C: 1, gamma: 0.01
Acurácia: 0.50, Precision: 0.35, Recall: 0.50, F1-Score: 0.41



...

c:\Users\Pichau\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Kernel: rbf, C: 10, gamma: 0.001
Acurácia: 0.43, Precision: 0.19, Recall: 0.43, F1-Score: 0.26



c:\Users\Pichau\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Kernel: rbf, C: 10, gamma: 0.01
Acurácia: 0.43, Precision: 0.19, Recall: 0.43, F1-Score: 0.26

Kernel: rbf, C: 10, gamma: 0.1
Acurácia: 0.43, Precision: 0.19, Recall: 0.43, F1-Score: 0.26



c:\Users\Pichau\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
